# Scenario D - Peakshape Variation (multiple runs)

In this scenario the peakshape in a spectrum with a fixed number of peaks is varied from Gaussian (n = 0.0) to Lorentzian (n = 1.0). All datasets contain 3 peaks and the noise level is kept constant at 1%.

The model used in the inference of the parameters is formulated as follows:

\begin{equation}
\large y = f(x) = \sum\limits_{m=1}^M \big[A_m \cdot f_{pseudo-Voigt}(x)\big] + \epsilon
\end{equation}

where:

\begin{equation}
\large f_{pseudo-Voigt}(x) = \eta \cdot \frac{\sigma_m^2}{(x-\mu_m)^2 + \sigma_m^2} + (1 - \eta) \cdot e^{-\frac{(x-\mu_m)^2}{2\cdot\sigma_m^2}}
\end{equation}

This file runs a series of inference runs for a set of generated spectra. New spectra are generated for each run and stored. After running inference, only the summary statistics are stored and the next run is started.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az

#az.style.use('arviz-darkgrid')

print('Running on PyMC3 v{}'.format(pm.__version__))

## Import local modules

In [ ]:
import os
import sys
sys.path.append('../../modules')
import datagen as dg
import models as mdl
import results as res
import figures as fig
import settings as cnf

## Local configuration

In [ ]:
# output for results and images
out_path      = './output_peakshape_mruns'
file_basename = out_path + '/scenario_peakshape'

conf = {}
    
# scenario name
conf['scenario'] = 'peakshape variation'
    
# initialization method for sampler
conf['init_mode'] = 'adapt_diag'

# probabilistic model (priors)
conf['prior_model'] = 'lognormal'

# provide peak positions to the model as testvalues ('yes'/'no')
conf['peak_info'] = 'yes'

# data mode ('generate'/'preload')
conf['data_mode'] = 'generate'

# number of runs
conf['nruns'] = 8

# number of cores to run sampling chains on
conf['ncores'] = 2

# number of samples per chain
conf['nsamples'] = 2000

In [ ]:
# if the output dir does not exist, create it
if not os.path.exists(out_path):
    os.makedirs(out_path)

conf

## Save configuration

In [ ]:
cnf.save(out_path, conf)

# Generate data and plot

In [ ]:
# list of wavelengths (x-values)
xval = [i for i in range(200, 400, 2)]

# number of spectra per peak number
nsets  = 4

# peakshape weight factors (0 = Gauss, 1 = Lorentz)
peakshapes = [0.0, 0.25, 0.5, 0.75, 1.0]
lpeakshape = [ps for ps in peakshapes for i in range(nsets)]

# total number of datasets
tsets = nsets * len(peakshapes)

# total number of inference runs (per run)
truns = nsets * len(peakshapes)**2

# generate nruns sets of spectra
for r in range(conf['nruns']):
    print("Generating dataset {0} of {1}".format(r+1,conf['nruns']))
    
    ldata, lpeaks = [], []
    
    # create output directory for data
    out_dir = out_path + '/run_{0:02d}'.format(r+1)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
            
    for ps in peakshapes:
        for i in range(nsets):
            df, peaks, _ = dg.data_generator(xvalues=xval, nsamples=15, npeaks=3, peakshape=ps)
            ldata.append(df)
            lpeaks.append(peaks)
            
    # save data and peak information to disk
    for i in range(len(ldata)):
        ldata[i].to_csv(out_dir + '/dataset_{0:02d}.csv'.format(i+1), index=False)
    dg.data_save(out_dir + '/peakinfo.csv', lpeaks)
    
    # plot datasets
    filen = out_dir + '/scenario_baseline'
    fig.plot_datasets(ldata, lpeaks, dims=(int(tsets/2),2), figure_size=(12,int(tsets*(1.8))), 
                                            savefig='yes', fname=filen, scenario='peakshape', labels=lpeakshape)

In [ ]:
print("total number of multiple runs                    : {0}".format(conf['nruns']))
print("total number of peakshape variations             : {0}".format(len(peakshapes)))
print("total number of datasets per peakshape           : {0}".format(nsets))
print("total number of datasets per model               : {0}".format(tsets))
print("total number of inference runs (per single loop) : {0}".format(truns))

# Load data, run inference, visualize, collect results and save 

In [ ]:
# convert pandas data to numpy arrays
x_val = np.array(xval, dtype='float32')

In [ ]:
# dataframe to hold multiple run results
res_df = pd.DataFrame()

# run the whole loop of inference, posterior sampling, results collection and saving
for r in range(conf['nruns']):
    print("starting loop {0}/{1}".format(r+1,conf['nruns']))

    models, traces, lmodbase = [], [], []

    # load datasets from disk
    data_dir = out_path + '/run_{0:02d}'.format(r+1)
    ldata, lpeaks, _ = dg.data_load(tsets, data_dir)

    # store dataset y-values in list
    cols = ldata[0].columns
    y_val = [ldata[i][cols].values for i in range(len(ldata))]

    # actual inference run number
    inf_run = 1

    for ps in peakshapes:
        print("running: baseline-{0} model".format(ps))
        for i in range(len(ldata)):
            if conf['peak_info'] == 'yes':
                plist = np.array(lpeaks[i], dtype=float).flatten()
                plist.sort()
                model_g = mdl.model_pvoigt(xvalues=x_val, observations=y_val[i], npeaks=3, peakshape=ps,
                                  mu_peaks=plist, pmodel=conf['prior_model'])
            else:
                model_g = mdl.model_pvoigt(xvalues=x_val, observations=y_val[i], npeaks=3, peakshape=ps,
                                                      pmodel=conf['prior_model'], baseline=bl)  
            models.append(model_g)

            with model_g:
                print("({6}:{2}/{3}) running inference on dataset #{0}/{1} [{4} model: {5} data]"
                      .format(i+1,len(ldata),inf_run,truns,ps,lpeakshape[i],r+1))
                trace_g = pm.sample(conf['nsamples'], init=conf['init_mode'], cores=conf['ncores'])
                lmodbase += [(ps,lpeakshape[i])]
                traces.append(trace_g)
                inf_run += 1

    # save model figure as image
    img = pm.model_to_graphviz(models[0])
    img.render(filename=file_basename + '_model', format='png');

    # posterior predictive traces
    ppc = [pm.sample_posterior_predictive(traces[i], samples=500, model=models[i]) for i in range(len(traces))]

    # collect the results, concat single run result to overall result
    varnames = mdl.get_varnames(traces[0])
    lruns = ['{0}'.format(r+1) for i in range(truns)]
    df = res.get_results_summary(traces, ppc, y_val, varnames, epsilon_real=0.05, 
                                 sets=tsets, labels=lmodbase, runlist=lruns)
    
    res_df = res_df.append(df, ignore_index=True)

## Show results and save

In [ ]:
res_df

In [ ]:
# save results to .csv
res_df.to_csv(out_path + '/scenario_baseline_mruns.csv', index=False)

In [ ]:
cnf.close(out_path)